# Deep Learning
- Application
  - Lots of data
  - Complex problems to solve
- Advantage
  - Adaptability
  - **Common infrastructure**
  - **Common language**
- From neural networks
  - 2009 speech recognition
  - 2010 computer vision
  - 2014 machine translation
- Drivers
  - Lots of data
  - GPUs

# Install TensorFlow

## Conda
```sh
conda create --name=IntroToTensorFlow python=3 anaconda
source activate IntroToTensorFlow
conda install -c conda-forge tensorflow
```

## Docker
```sh
docker run -it -p 8888:8888 gcr.io/tensorflow/tensorflow
```

# Hello World

In [1]:
import tensorflow as tf

# Create TensorFlow object called tensor
hello_constant = tf.constant('Hello World!')

with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


- In tensorflow, data are encapsulated in an object called a tensor.
  - `hello_constant` is a 0-dimensional string tensor
  - `tf.constant(1234)` 0-dimensional int32 tensor
  - `tf.constant([1234, 5678])` 1-dimensional int32 tensor
  - `tf.constant([ [123,456,789], [222,333,444] ])` 2-dimensional int32 tensor

- A tensorflow session is an environment for running a graph
  - In charge of allocating the operations to GPU(s) and/or CPU(s), including remote machines
  <img src="images/tensorflow-session.png" width="50%">
  - The code creates a session instance, `sess`, using `tf.Session`.
  - `sess.run()` evaluates the tensor and returns the results.

# TensorFlow Input
- `tf.placeholder()` returns a tensor that gets its value from data passed to `tf.sesssion.run()`.
- `feed_dict` parameter in `tf.session.run()` sets the placeholder tensor.

In [2]:
import tensorflow as tf

x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'String', y: 123, z: 45.6})
    print(output)

String


# TensorFlow Math
- Operations: `tf.add`, `tf.subtract`, `tf.multiply`, `tf.divide`
- Types must match
- Type cast: `tf.cast(<variable>, tf.<type>)` 
- Matrix multiplication: `tf.matmul`

# Logistic classifier
- Linear classifier
  - $ y = Wx + b $

# TensorFlow Variable
- `tf.Variable()`
  - Create a tensor with an initial value that can be modified
  - Store its state in the session
  - Use `tf.global_variables_initializer()` to initialize the state of all `Variable` tensors
  ```py
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
      sess.run(init)
  ```
- `tf.truncated_normal()`
  - Return a tensor with random values from a normal distribution whose magnitude is no more than 2 standard deviations from the mean
  - Since the weights are already helping prevent the model from getting stuck, you don't need to randomize the bias. Let's use the simplest solution, setting the bias to 0
  ```py
  n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))
  ```
- `tf.zeros()`
  - Return a tensor with all zeros
  ```py
  n_labels = 5
bias = tf.Variable(tf.zeros(n_labels))
  ```

# TensorFlow Linear Function

In [10]:
import tensorflow as tf

def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # Return weights
    return tf.Variable(tf.truncated_normal((n_features, n_labels)))


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # Return biases
    return tf.Variable(tf.zeros(n_labels))


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # Linear Function (xW + b)
    return tf.add(tf.matmul(input, w), b)

In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('datasets/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels


# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels
n_labels = 3

# Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

# Weights and Biases
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

# Linear Function xW + b
logits = linear(features, w, b)

# Training data
train_features, train_labels = mnist_features_labels(n_labels)

with tf.Session() as session:
    # Initialize session variables
    session.run(tf.global_variables_initializer())

    # Softmax
    prediction = tf.nn.softmax(logits)

    # Cross entropy
    # This quantifies how far off the predictions were.
    # You'll learn more about this in future lessons.
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

    # Training loss
    # You'll learn more about this in future lessons.
    loss = tf.reduce_mean(cross_entropy)

    # Rate at which the weabights are changed
    # You'll learn more about this in future lessons.
    learning_rate = 0.08

    # Gradient Descent
    # This is the method used to train the model
    # You'll learn more about this in future lessons.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Run optimizer and get loss
    _, l = session.run(
        [optimizer, loss],
        feed_dict={features: train_features, labels: train_labels})

# Print loss
print('Loss: {}'.format(l))


Extracting datasets/mnist/train-images-idx3-ubyte.gz
Extracting datasets/mnist/train-labels-idx1-ubyte.gz
Extracting datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting datasets/mnist/t10k-labels-idx1-ubyte.gz
Loss: 5.569997310638428


# TensorFlow Softmax